## Resampling with ODC-geo (local storage, NetCDF File, H5NetCDF driver)

In [ ]:
import argparse
import sys

import fsspec
import xarray as xr
from odc.geo.geobox import GeoBox
from odc.geo.xr import xr_reproject

In [ ]:
sys.path.append("..")

In [ ]:
def warp_resample(dataset):
    from common import earthaccess_args
    from common import target_extent as te

    # Define filepath, driver, and variable information
    args = earthaccess_args[dataset]
    src = f'../earthaccess_data/{args["filename"]}'
    # Define source and target projection
    dstSRS = "EPSG:3857"
    srcSRS = "EPSG:4326"
    width = height = 256
    # Specify fsspec caching since default options don't work well for raster data
    fsspec_caching = {
        "cache_type": "none",
    }
    fs = fsspec.filesystem("file")
    with fs.open(src, **fsspec_caching) as f:
        # Define ODC geobox for target tile
        gbox = GeoBox.from_bbox(te, dstSRS, shape=(height, width))
        # Open dataset
        da = xr.open_dataset(f, engine="h5netcdf")[args["variable"]]
        if dataset == "gpm_imerg":
            # Transpose and rename dataset dims to align with GDAL expectations
            da = (
                da.rename({"lon": "x", "lat": "y"})
                .transpose("time", "y", "x")
                .squeeze()
            )
        # Assign input projection
        da = da.odc.assign_crs(srcSRS)
        # Reproject dataset
        return xr_reproject(da, gbox)

In [ ]:
if __name__ == "__main__":
    if "get_ipython" in dir():
        # Just call warp_resample if running as a Jupyter Notebook
        da = warp_resample("gpm_imerg")
    else:
        # Configure dataset via argpase if running via CLI
        parser = argparse.ArgumentParser(description="Set environment for the script.")
        parser.add_argument(
            "--dataset",
            default="mursst",
            help="Dataset to resample.",
            choices=["gpm_imerg", "mursst"],
        )
        user_args = parser.parse_args()
        da = warp_resample(user_args.dataset)